In [1]:
# Importing modules & setting up pass
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

········


In [2]:
# How many times a movie was rented? sakila.film vs sakila.rental
# Which category of movies in most rented? sakila.category vs sakila.rental vs sakila.film_category
# Which rating of movies is most rented? sakila.rating vs sakila.rental
# Has the length of the movie some influence? sakila.film vs sakila.rental
# What about the language? sakila.language vs sakila.film
# Special-features? sakila.film
# Rental price/rate? sakila.film
# Rental duration? sakila.film

In [3]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [4]:
sakila =  '''
SELECT f.film_id, COUNT(r.rental_id) rented_times,f.rental_duration, c.name as category, f.length,
f.rating, f.special_features, f.rental_rate, l.name AS language,
CASE
    WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN True
    ELSE False END AS may
FROM sakila.film f
Left JOIN inventory i
    ON f.film_id = i.film_id
JOIN sakila.rental r
    ON i.inventory_id = r.inventory_id
Join sakila.film_category fc
    On fc.film_id = f.film_id
Join sakila.category c
    On c.category_id = fc.category_id
Join sakila.language l
    On l.language_id = f.language_id
Where r.rental_date Between '2005-01-01' AND '2005-12-31'
GROUP BY film_id, rental_duration,category, f.length, f.rating, may, f.special_features,f.rental_rate, language; 
'''


sakila = pd.read_sql_query(sakila, engine)

In [5]:
sakila

,film_id,rented_times,rental_duration,category,length,rating,special_features,rental_rate,language,may
0,19,19,6,Action,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",0.99,English,0
1,19,1,6,Action,113,PG,"Commentaries,Deleted Scenes,Behind the Scenes",0.99,English,1
2,21,2,3,Action,129,R,"Commentaries,Behind the Scenes",4.99,English,1
3,21,19,3,Action,129,R,"Commentaries,Behind the Scenes",4.99,English,0
4,29,10,5,Action,168,NC-17,"Trailers,Commentaries,Deleted Scenes",2.99,English,0
...,...,...,...,...,...,...,...,...,...,...
1580,977,11,3,Travel,85,R,"Deleted Scenes,Behind the Scenes",2.99,English,0
1581,981,21,7,Travel,55,NC-17,Behind the Scenes,0.99,English,0
1582,988,15,7,Travel,139,R,"Trailers,Commentaries,Behind the Scenes",2.99,English,0
1583,989,23,4,Travel,74,R,"Commentaries,Deleted Scenes",4.99,English,0


In [8]:
sakila.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           1585 non-null   int64  
 1   rented_times      1585 non-null   int64  
 2   rental_duration   1585 non-null   int64  
 3   category          1585 non-null   object 
 4   length            1585 non-null   int64  
 5   rating            1585 non-null   object 
 6   special_features  1585 non-null   object 
 7   rental_rate       1585 non-null   float64
 8   language          1585 non-null   object 
 9   may               1585 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 124.0+ KB


One hot encoding / Get dummies / Whatever! the special_features...

In [ ]:
sakila['special_features'].value_counts()

In [9]:
#this can be improved...this is a lot of code to a simple task....
trailer = []; b_scenes = []; commentaries = []; s_features = []

for i in sakila['special_features']:
        if re.findall('Trailers', i):
            trailer.append(1)
        else:
            trailer.append(0)
for i in sakila['special_features']:
        if re.findall('Behind the Scenes', i):
            b_scenes.append(1)
        else:
            b_scenes.append(0)      
for i in sakila['special_features']:
        if re.findall('Commentaries', i):
            commentaries.append(1)
        else:
            commentaries.append(0)     
for i in sakila['special_features']:
        if re.findall('Deleted Scenes', i):
            s_features.append(1)
        else:
            s_features.append(0)
            
sakila['trailers'] = trailer
sakila['commentaries'] = commentaries
sakila['behind the scenes'] = b_scenes  
sakila['deleted scenes'] = s_features

In [10]:
sakila = sakila.drop(['special_features'], axis=1)
#there's no need for it now

In [12]:
# as objects
sakila['film_id'] = sakila['film_id'].astype(object)
sakila['rental_duration'] = sakila['rental_duration'].astype(object)
sakila['rental_rate'] = sakila['rental_rate'].astype(object)
sakila['language'] = sakila['language'].astype(object)
# as bools
sakila['trailers'] = sakila['trailers'].astype(bool)
sakila['may'] = sakila['may'].astype(bool)
sakila['behind the scenes'] = sakila['behind the scenes'].astype(bool)
sakila['commentaries'] = sakila['commentaries'].astype(bool)
sakila['deleted scenes'] = sakila['deleted scenes'].astype(bool)

In [ ]:
y = sakila['may']
X = sakila.drop('may', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)